In [ ]:
import matplotlib
import numpy
import json
import dataclasses
import os
import csv

@dataclasses.dataclass
class Answer:
	eye_shape : str
	eyebrow : str
	iris : str
	pupil : str

@dataclasses.dataclass
class Data:
	id : str
	answers : list[Answer]


EYE_SHAPE_TR = {
	"?"		: "Eye4",
	"??"	: "Eye5",
	"???"	: "Eye6",
	"????"	: "Eye2",
	"?????" : "Eye1",
	"??????": "Eye3",
}

IRIS_TR = {
	"1" : "Iris1",
	"2" : "Iris4",
	"3" : "Iris6",
	"4" : "Iris5",
	"5" : "Iris3",
	"6" : "Iris2",
}

PUPIL_TR = {
	"1" : "Pupil4",
	"2" : "Pupil6",
	"3" : "Pupil5",
	"4" : "Pupil3",
	"5" : "Pupil1",
	"6" : "Pupil2",
}

EYEBROW_TR = {
	"1" : "Eyebrow4",
	"2" : "Eyebrow5",
	"3" : "Eyebrow6",
	"4" : "Eyebrow2",
	"5" : "Eyebrow3",
	"6" : "Eyebrow1",
}


def load_main_questionnaire(path = "./main_questionnaire.csv")->list[Data]:
	ret :list[Data] = []
	with open(path, "r", encoding="utf-8") as csvfile:
		is_first_line = True
		for row in csv.reader(csvfile):
			if is_first_line:
				is_first_line = False
				continue
			current_data = Data(0, [])
			current_data.id = row[2]
			for i in range(6, len(row), 4):
				current_data.answers.append(Answer(
					EYE_SHAPE_TR[row[i]],
					IRIS_TR[row[i+1]],
					PUPIL_TR[row[i+2]],
					EYEBROW_TR[row[i+3]],
				))
			ret.append(current_data)
	return ret


def load_log_raw(path):
	with open(path, "r") as f:
		return json.load(f)
	

def load_log(path):
	raw = load_log_raw(path)
	ret = Data(0, [])
	for entry in raw:
		entry_type = entry['type']
		if entry_type == 'user_id':
			ret.id = entry['id']
		if entry_type == 'submit_answer':
			ret.answers.append(Answer(
				entry['eye_shape'],
				entry['eyebrow'],
				entry['iris'],
				entry['pupil'],
			))
	return ret

def load_logs(dir_path = "./game_logs"):
	ret :list[Data] = []
	for path in os.scandir(dir_path):
		if path.is_file():
			ret.append(load_log(path.path))
	return ret


In [29]:
load_main_questionnaire()

KeyError: 'Eye shape'